# Word Count Example

In [ ]:
!mkdir -p /root/python_mr/data
!hdfs dfs -mkdir /user/ashu/

In [ ]:
!wget http://www.gutenberg.org/cache/epub/4300/pg4300.txt
!wget http://www.gutenberg.org/cache/epub/5000/pg5000.txt

In [ ]:
!hdfs dfs -put /root/python_mr/data/* /user/ashu

In [1]:
!mkdir data

In [3]:
!ls

data  data.ipynb


In [ ]:
# %load count_mapper.py
#!/usr/bin/env python

import sys
for line in sys.stdin:
    line = line.strip()
    words = line.split()
    for word in words:
        print '%s\t%s' % (word, 1)


In [ ]:
# %load count_reducer.py
#!/usr/bin/env python
import sys

current_word = None
current_count = 0
word = None

for line in sys.stdin:
    word, count = line.split('\t')
    count = int(count)
    if word == current_word:
        current_count += count
    else:
        if current_word:
            print '%s\t%i' % (current_word, current_count)
        current_word = word
        current_count = count

if current_word == word:
    print '%s\t%i' % (current_word, current_count)
    
    

In [25]:
!chmod +x count_*.py

In [3]:
!echo "foo foo quux labs foo bar quux" | /root/python_mr/word_count/count_mapper.py

foo	1
foo	1
quux	1
labs	1
foo	1
bar	1
quux	1


In [4]:
!echo "foo foo quux labs foo bar quux" | /root/python_mr/word_count/count_mapper.py |sort | /root/python_mr/word_count/count_reducer.py 

bar	1
foo	3
labs	1
quux	2


In [11]:
! hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.2.jar \
-file /root/python_mr/word_count/count_mapper.py -mapper /root/python_mr/word_count/count_mapper.py \
-file  /root/python_mr/word_count/count_reducer.py -reducer  /root/python_mr/word_count/count_reducer.py \
-input /user/ashu/* \
-output /user/ashu/book-output

16/03/22 15:22:07 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
16/03/22 15:22:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/root/python_mr/word_count/count_mapper.py, /root/python_mr/word_count/count_reducer.py, /tmp/hadoop-unjar5744367187597170740/] [] /tmp/streamjob2603901880557568741.jar tmpDir=null
16/03/22 15:22:08 INFO client.RMProxy: Connecting to ResourceManager at /10.211.55.101:8032
16/03/22 15:22:09 INFO client.RMProxy: Connecting to ResourceManager at /10.211.55.101:8032
16/03/22 15:22:10 INFO mapred.FileInputFormat: Total input paths to process : 2
16/03/22 15:22:10 INFO mapreduce.JobSubmitter: number of splits:3
16/03/22 15:22:10 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1458650496725_0002
16/03/22 15:22:10 INFO impl.YarnClientImpl: Submitted application application_1458650496725_0002
16/03/22 15:22:11

In [33]:
! hdfs dfs -ls -h /user/ashu/book-output

16/03/21 18:46:20 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 root supergroup          0 2016-03-21 18:45 /user/ashu/book-output/_SUCCESS
-rw-r--r--   3 root supergroup      1.0 M 2016-03-21 18:45 /user/ashu/book-output/part-00000


In [10]:
! hdfs dfs -rm -r /user/ashu/book-output

16/03/22 15:22:03 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/03/22 15:22:05 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/ashu/book-output


In [ ]:
# !hdfs dfs -cat /user/ashu/book-output/part-00000

reference

[blog](http://www.quuxlabs.com/tutorials/writing-an-hadoop-mapreduce-program-in-python/)<br>
[github](https://github.com/codeAshu/hadoop-python-hive-tutorial/blob/master/mapreduce.md)